Download----Prepare-------Train and Interpret-------Overfitting..----Hyperparameter Tuning---Make PRedictions on Single inputs

In [1]:
!pip install opendatasets scikit-learn jovian --quiet --upgrade

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 1.3 MB/s eta 0:00:00


In [2]:
import opendatasets as od
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import jovian
import os
%matplotlib inline

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',150)
sns.set_style('darkgrid')
matplotlib.rcParams['font.size']=14
matplotlib.rcParams['figure.figsize'] =(10,6)
matplotlib.rcParams['figure.facecolor'] ='#00000000'


In [3]:
od.download('https://www.kaggle.com/jsphyg/weather-dataset-rattle-package')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

Abort: 

In [ ]:
os.listdir('weather-dataset-rattle-package')

In [ ]:
raw_df = pd.read_csv('weather-dataset-rattle-package/weatherAUS.csv')

In [ ]:
raw_df

In [ ]:
raw_df.info()

In [ ]:
raw_df.dropna(subset=['RainTomorrow'],inplace=True)

In [ ]:
raw_df.info()

Preparing the Data for Training
We'll perform the following steps to prepare the dataset for training:

Create a train/test/validation split
Identify input and target columns
Identify numeric and categorical columns
Impute (fill) missing numeric values
Scale numeric values to the
(
0
,
1
)
(0,1) range
Encode categorical columns to one-hot vectors

In [ ]:
plt.title('No. of Rows per Year')
sns.countplot(x=pd.to_datetime(raw_df.Date).dt.year);

In [ ]:
year =pd.to_datetime(raw_df.Date).dt.year

train_df = raw_df[year <2015]
val_df =raw_df[year ==2015]
test_df = raw_df[year >2015]


In [ ]:
print('train_df.shape:',train_df.shape)
print('val_df.shape:',val_df.shape)
print('test_df.shape:',test_df.shape)

In [ ]:
input_cols = list(train_df.columns)[1:-1]
target_col = 'RainTomorrow'

In [ ]:
train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()

In [ ]:
val_inputs = train_df[input_cols].copy()
val_targets =val_df[target_col].copy()

In [ ]:
test_inputs = test_df[input_cols].copy()
test_targets = test_df[target_col].copy()

In [ ]:
numeric_cols =train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols =train_inputs.select_dtypes('object').columns.tolist()

In [ ]:
print(numeric_cols)

In [ ]:
print(categorical_cols)

Impute to Discard Imbalance of Data

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy = 'mean').fit(raw_df[numeric_cols])

In [ ]:
train_inputs[numeric_cols] = imputer.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] =imputer.transform(val_inputs[numeric_cols])
test_inputs[numeric_cols] =imputer.transform(test_inputs[numeric_cols])

In [ ]:
test_inputs[numeric_cols].isna().sum()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler().fit(raw_df[numeric_cols])


In [ ]:
train_inputs[numeric_cols] = scaler.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] = scaler.transform(val_inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])


In [ ]:
val_inputs.describe().loc[['min','max']]

For the Categorical Values to convert and Encode

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder = OneHotEncoder(sparse_output = False,handle_unknown='ignore').fit(raw_df[categorical_cols])

In [ ]:
encoded_cols =list(encoder.get_feature_names_out(categorical_cols))

In [ ]:
encoded_cols

In [ ]:
train_inputs[encoded_cols] = encoder.transform(train_inputs[categorical_cols])
val_inputs[encoded_cols] = encoder.transform(val_inputs[categorical_cols])
test_inputs[encoded_cols] = encoder.transform(test_inputs[categorical_cols])


In [ ]:
test_inputs

In [ ]:
X_train =train_inputs[numeric_cols +encoded_cols]
X_val = val_inputs[numeric_cols +encoded_cols]
X_test = test_inputs[numeric_cols +encoded_cols]

In [ ]:
X_test

Train and Visual

In [ ]:
from sklearn.tree import DecisionTreeClassifier


In [ ]:
model = DecisionTreeClassifier(random_state =42)

In [ ]:
%%time
model.fit(X_train,train_targets)

evaluation`

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
train_preds = model.predict(X_train)

In [ ]:
train_preds

In [ ]:
pd.value_counts(train_preds)

In [ ]:
train_probs =model.predict_proba(X_train)

In [ ]:
train_probs

In [ ]:
accuracy_score(train_targets,train_preds)

In [ ]:
X_val

In [ ]:
model.score(X_val,val_targets)

In [ ]:
val_targets.value_counts()/len(val_targets)

In [ ]:
from sklearn.tree import plot_tree,export_text

In [ ]:
plt.figure(figsize=(80,20))
plot_tree(model,feature_names =X_train.columns,max_depth=2,filled=True);

In [ ]:
model.tree_.max_depth

In [ ]:
tree_text =export_text(model,max_depth=10,feature_names=list(X_train.columns))
print(tree_text[:5000])

In [ ]:
model.feature_importances_

In [ ]:
importance_df=pd.DataFrame({
    'feature':X_train.columns,
    'importance':model.feature_importances_
}).sort_values('importance',ascending=False)

In [ ]:
importance_df.head(10)

In [ ]:
plt.title('Feature Importance')
sns.barplot(data=importance_df.head(10),x = 'importance',y='feature');

In [ ]:
?DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier(max_depth =3,random_state=42)

In [ ]:
model.fit(X_train,train_targets)

In [ ]:
model.score(X_train,train_targets)

In [ ]:
model.score(X_val,val_targets)

In [ ]:
model.classes_

In [ ]:
plt.figure(figure=(80,20))
plot_tree(model, feature_names=X_train.columns, filled=True, rounded=True, class_names=model.classes_);


In [ ]:
print(export_text(model,feature_names=list(X_train.columns)))

In [ ]:
def max_depth_error(md):
  model =DecisionTreeClassifier(max_depth=md,random_state=42)
  model.fit(X_train,train_targets)
  train_acc = 1-model.score(X_train,train_targets)
  val_acc = 1-model.score(X_val,val_targets)


In [ ]:
%%time
errors_df = pd.DataFrame([max_depth_error(md) for md in range(1,2)])

In [ ]:
errors_df

In [ ]:
plt.figure()
plt.plot(errors_df['Max Depth'],errors_df['Training Error'])
plot.plot(errors_df['Max Depth'],errors_df['validation Error'])
plt.title('Training vs. Validation Error')
plt.xticks(range(0,51,2))
plt.xlabel('Max,Depth')
plt.ylabel('Prediction Error(1-Accuracy)')
plt.legend(['Training','Validation'])

In [ ]:
model = DecisionTreeClassifier(max_depth = 7,random_state =42).fit(X_train,train_targets)
model.score(X_val,val_targets)

In [ ]:
model.fit(X_train,train_targets)

In [ ]:
model.score(X_val,val_targets)

In [ ]:
model.tree_.max_depth

In [ ]:
model_text =export_text(model,feature_names=list(X_train.columns))
print(model_text[:3000])

Training RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_jobs =-1,random_state=42)

In [ ]:
%%time
model.fit(X_train,train_targets)

In [ ]:
model.fit(X_train,train_targets)

In [ ]:
model.score(X_val,val_targets)

In [ ]:
train_probs = model.predict_proba(X_train)
train_probs

In [ ]:
model.estimators_[0]

In [ ]:
plt.figure(figsize=(80,20))
plot_tree(model.estimators_[0],max_depth=2,feature_names=X_train.columns,filled=True,rounded=True,class_names=model.classes_);

In [ ]:
plt.figure(figsize=(80,20))
plot_tree(model.estimatoes_[20],max_dapth=2,feature_names=X_train.columns,filled=True,rounded=True,class_names=model.classes_);

In [ ]:
len(model.estimators_)

In [ ]:
importance_df = pd.DataFrame({
    'feature':X_train.columns,
    'importance':model.feature_importances_
}).sort_values('importance',ascending=False)

In [ ]:
importance_df.head(10)

In [ ]:
plt.title('Feature Importance')
sns.barplot(data=importance_df.head(10), x='importance', y='feature');

In [ ]:
?RandomForestClassifier

In [ ]:
base_model =RandomForestClassifier(random_state =42,n_jobs=-1).fit(X_train,train_targets)

In [ ]:
base_train_acc =base_model.score(X_train,train_targets)
base_val_acc =base_model.score(X_val,val_targets)

In [ ]:
base_accs = base_train_acc,base_val_acc
base_accs

N+Estimators

In [ ]:
model =RandomForestClassifier(random_state=42,n_jobs=-1,n_estimator=10)

In [ ]:
model.fit(X_train,train_targets)

In [ ]:
model.score(X_train,train_targets),model.score(X_val,val_targets)

In [ ]:
base_accs

In [ ]:
model =RandomForestClassifier(random_state=42,n_jobs=-1,n_esimators=500)
model.fit(X_train,train_targets)

In [ ]:
model.score(X_train,train_targets)

In [ ]:
model.score(X_val,val_targets)

In [ ]:
base_accs

In [ ]:
def test_params(**params):
  model=RandomForestClassifier(random_state=42,n_jobs=-1,**params).fit(X_train,train_targets)
  return model.score(X_train,train_targets),model.score(X_val,val_targets)


In [ ]:
test_params(max_depth=5)

In [ ]:
test_params(max_params=26)

In [ ]:
test_params(max_leaf_nodes=2**5)

In [ ]:
test_params(max_leaf_nodes=2**20)

In [ ]:
base_accs

### `max_features`

Instead of picking all features (columns) for every split, we can specify that only a fraction of features be chosen randomly to figure out a split.

<img src="https://i.imgur.com/FXGWMDY.png" width="720">

Notice that the default value `auto` causes only $\sqrt{n}$ out of total features ( $n$ ) to be chosen randomly at each split. This is the reason each decision tree in the forest is different. While it may seem counterintuitive, choosing all features for every split of every tree will lead to identical trees, so the random forest will not generalize well.

In [ ]:
test_params(max_features ='log2')

In [ ]:
test_params(max_features=3)

In [ ]:
test_params(max_features=6)

In [ ]:
base_accs

### `min_impurity_decrease`

This argument is used to control the threshold for splitting nodes. A node will be split if this split induces a decrease of the impurity (Gini index) greater than or equal to this value. It's default value is 0, and you can increase it to reduce overfitting.


In [ ]:
test_params(min_impurity_decrease=1e-7)

In [ ]:
test_params(min_impurity_decrease=1e-2)

In [ ]:
base_accs

### `bootstrap`, `max_samples`

By default, a random forest doesn't use the entire dataset for training each decision tree. Instead it applies a technique called bootstrapping. For each tree, rows from the dataset are picked one by one randomly, with replacement i.e. some rows may not show up at all, while some rows may show up multiple times.


<img src="https://i.imgur.com/W8UGaEA.png" width="640">

Bootstrapping helps the random forest generalize better, because each decision tree only sees a fraction of th training set, and some rows randomly get higher weightage than others.

In [ ]:
test_params(bootstrap=False)

In [ ]:
base_accs

In [ ]:
model_classes_

In [ ]:
test_params(class_weights='balanced')

In [ ]:
test_params(class_weight={'No':1,'yes':2})

In [ ]:
base_accs

When bootstrapping is enabled, you can also control the number or fraction of rows to be considered for each bootstrap using `max_samples`. This can further generalize the model.

<img src="https://i.imgur.com/rsdrL1W.png" width="640">

In [ ]:
test_params(max_samples=0.9)

In [ ]:
base_accs

In [ ]:
model.classes_

In [ ]:
test_params(class_weight='balanced')

In [ ]:
test_params(class_wright={'No':1,'Yes':2})

In [ ]:
base_accs

In [ ]:
model = RandomForestClassifier(n_jobs =-1,
                               random_stat=42,
                               n_estimators=500,
                               max_features=7,
                               max_depth=30,
                               class_weight={'No':1,'Yes':1.5})

In [ ]:
model.fit(X_train,train_targets)

In [ ]:
model.score(X_train,train_targets),model.score(X_val,val_targets)

In [ ]:
base_accs

We've increased the accuracy from 84.5% with a single decision tree to 85.7% with a well-tuned random forest. Depending on the dataset and the kind of problem, you may or may not a see a significant improvement with hyperparameter tuning.

This could be due to any of the following reasons:

- We may not have found the right mix of hyperparameters to regularize (reduce overfitting) the model properly, and we should keep trying to improve the model.

- We may have reached the limits of the modeling technique we're currently using (Random Forests), and we should try another modeling technique e.g. gradient boosting.

- We may have reached the limits of what we can predict using the given amount of data, and we may need more data to improve the model.

- We may have reached the limits of how well we can predict whether it will rain tomorrow using the given weather measurements, and we may need more features (columns) to further improve the model. In many cases, we can also generate new features using existing features (this is called feature engineering).

- Whether it will rain tomorrow may be an inherently random or chaotic phenomenon which simply cannot be predicted beyond a certain accuracy any amount of data for any number of weather measurements with any modeling technique.  

Remember that ultimately all models are wrong, but some are useful. If you can rely on the model we've created today to make a travel decision for tomorrow, then the model is useful, even though it may sometimes be wrong.

In [ ]:
raw_df

In [ ]:
model.score(X_test,test_targets)

In [ ]:
def predict_input(model,single_input):
  input_df =pd.DatFrame([single_input])
  input_df[numeric_cols] =imputer.transform(input_df[numeric_cols])
  input_df[numeric_cols] =scaler.transform(input_df[numeric_cols])
  input_df[encoded_cols] =encoder.transform(input_df[ctegorical_cols])
  X_input =input_df[numeric_cols +encoded+cols]
  pred =model.predict(X_input)[0]
  prob =model.predict_prob(X_input)[0][list(model.classes_).index(pred)]
  return pred,prob

In [ ]:
new_input={'Date':'2021006-19',
            'Location': 'Launceston',
             'MinTemp': 23.2,
             'MaxTemp': 33.2,
             'Rainfall': 10.2,
             'Evaporation': 4.2,
             'Sunshine': np.nan,
             'WindGustDir': 'NNW',
             'WindGustSpeed': 52.0,
             'WindDir9am': 'NW',
             'WindDir3pm': 'NNE',
             'WindSpeed9am': 13.0,
             'WindSpeed3pm': 20.0,
             'Humidity9am': 89.0,
             'Humidity3pm': 58.0,
             'Pressure9am': 1004.8,
             'Pressure3pm': 1001.5,
             'Cloud9am': 8.0,
             'Cloud3pm': 5.0,
             'Temp9am': 25.7,
             'Temp3pm': 33.0,
             'RainToday': 'Yes'
              }

In [ ]:
predict_input(model,new_input)

In [ ]:
raw_df.Location.unique()

In [ ]:
import joblib

In [ ]:
aussie_rain = {
    'model': model,
    'imputer': imputer,
    'scaler': scaler,
    'encoder': encoder,
    'input_cols': input_cols,
    'target_col': target_col,
    'numeric_cols': numeric_cols,
    'categorical_cols': categorical_cols,
    'encoded_cols': encoded_cols
}

In [ ]:
joblib.dump(aussie_rain, 'aussie_rain.joblib')

In [ ]:
aussie_rain2 = joblib.load('aussie_rain.joblib')

In [ ]:
test_preds2 = aussie_rain2['model'].predict(X_test)
accuracy_score(test_targets, test_preds2)